In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torch
import csv
import os
from torch.utils.data.sampler import SubsetRandomSampler

resume_weights = "/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/checkpointcrazy6.pth.tar"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import Python libraries  
import json
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from PIL import Image

# Import PyTorch libraries
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [ ]:
tforms = transforms.Compose([transforms.Grayscale(3),  
                             transforms.ToTensor(),
                             transforms.Normalize([0.458, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])])
test_transform = transforms.Compose([
       
        transforms.ToTensor(),
        transforms.Normalize([0.458, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])
    ])

In [ ]:
DEVICE_IDS = [0, 1, 2, 3]

In [ ]:
from torchvision import datasets
#dataFromFolders = datasets.ImageFolder(root='/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/train',transform = tforms)


import PIL
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
#len(dataFromFolders)
transforms2 = torchvision.transforms.Compose([
    #torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.Grayscale(3),  
    transforms.ToTensor(),
    transforms.Normalize([0.458, 0.456, 0.406],[0.229, 0.224, 0.225])])

#dataFromFolders2 = torchvision.datasets.ImageFolder('/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/train', transform=transforms)
# len(dataFromFolders2)
#dataFromFolders = torch.utils.data.ConcatDataset([dataFromFolders,dataFromFolders2])
# len(increased_dataset)
dataFromFolders = datasets.ImageFolder(root='/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/train',transform = transforms2)


# Creating data indices for training and validation splits:
batch_size = 16
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(dataFromFolders)
print(len(dataFromFolders))
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
folderloader = DataLoader(dataFromFolders, batch_size=batch_size,sampler=train_sampler)
print(folderloader)
images, labels = iter(folderloader).next()

#dataFromFolders_valid = datasets.ImageFolder(root='/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/train',transform = tforms)
folderloader_valid = DataLoader(dataFromFolders, batch_size=batch_size, sampler=valid_sampler)

dataFromFolders_test = datasets.ImageFolder(root='/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set',transform = test_transform)
folderloader_test = DataLoader(dataFromFolders_test,batch_size=1)

print(labels.size())
print(images.size())
print(len(folderloader))
print(len(folderloader_valid))
print(len(folderloader_test))

In [ ]:
# images, labels = iter(folderloader).next()
# #print(images)

In [ ]:
class AlexNet2(nn.Module):
    """
    Neural network model consisting of layers propsed by AlexNet paper.
    """
    def __init__(self, num_classes=7):
        """
        Define and allocate layers for this neural net.
        Args:
            num_classes (int): number of classes to predict with this model
        """
        super().__init__()
        # input size should be : (b x 3 x 227 x 227)  here (bx3x256x256)
        # The image in the original paper states that width and height are 224 pixels, but
        # the dimensions after first convolution layer do not lead to 55 x 55.
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  # (b x 96 x 55 x 55)   (bx96x62x62)
            nn.ReLU(True),
            nn.BatchNorm2d(64),
            #nn.Dropout(p=0.2, inplace=True),
            #nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),  # (b x 96 x 27 x 27)   (bx96x30x30)
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  # (b x 256 x 27 x 27)     (bx256x30x30)
            nn.ReLU(True),
            nn.BatchNorm2d(128),
            #nn.Dropout(p=0.2, inplace=True),
            #nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),  # (b x 256 x 13 x 13)  (bx256x14x14)
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  # (b x 384 x 13 x 13)  (bx512x14x14)
            nn.ReLU(True),
            nn.BatchNorm2d(256),
            nn.Dropout(p=0.2, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),  # (b x 96 x 27 x 27)   (bx96x30x30)
            nn.Conv2d(256,256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  # (b x 384 x 13 x 13)  (bx512x14x14)
            nn.ReLU(True),
            nn.BatchNorm2d(256),
            nn.Dropout(p=0.2, inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),  # (b x 96 x 27 x 27)   (bx96x30x30)
            # nn.Conv2d(512,512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),  # (b x 256 x 13 x 13)  (bx512x14x14)
            # nn.ReLU(True),
            # nn.BatchNorm2d(512),
            # nn.Dropout(p=0.2, inplace=True),
            # nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),  # (b x 256 x 6 x 6)   (bx256x6x6)
        )
        # classifier is just a name for linear layers
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=(256 * 6 * 6), out_features=512),
            nn.ReLU(True),
            nn.Dropout(p=0.5, inplace=True),
            # nn.Linear(in_features=4096, out_features=4096),
            #nn.ReLU(True),
            nn.Linear(in_features=512, out_features=num_classes),
            nn.LogSoftmax(dim=1)
        )
        #self.init_bias()  # initialize bias
        self._initialize_weights()

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[4].bias, 1)
        nn.init.constant_(self.net[10].bias, 1)
        nn.init.constant_(self.net[12].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        #x = self.classifier(x)
        #x = x.view(-1, 256 * 6 * 6)  # reduce the dimensions for linear layer input
        return self.classifier(x)
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [ ]:
best_accuracy = torch.FloatTensor([0])
start_epoch = 0

In [ ]:
# model = models.vgg11(pretrained=True)
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # model.to(device)
# display(model)

In [ ]:
torch.cuda.device_count()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model = AlexNet2(num_classes=7).to(device)
#model = torch.nn.parallel.DataParallel(model, device_ids=DEVICE_IDS)
display(model)

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:

# Parameters 
in_features = 256*7*7
out_categories = 7
layer_1 = 512
layer_2 = 512
# Build classifier
classifier = nn.Sequential(OrderedDict([
    #('dropout1', nn.Dropout(0.5)),
    ('fc1', nn.Linear(in_features, layer_1)),
    ('relu', nn.ReLU(True)),
    ('dropout2', nn.Dropout(0.5)),
    ('fc2', nn.Linear(layer_1, layer_2)),
    ('relu', nn.ReLU(True)),
    ('dropout3', nn.Dropout(0.5)),    
    ('fc3', nn.Linear(layer_2, out_categories)),
    ('output', nn.LogSoftmax(dim=1))
    ]))

# Assign new classifier to VGG-11 architecture 
model.classifier = classifier

In [ ]:

# Instantiate loss function
loss_function = nn.NLLLoss()

# Instantiate optimization algorithm 
learning_rate = 1e-4
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

# Enable CUDA: use GPUs for model computation
if torch.cuda.is_available():
    print('a')
    model.to('cuda')

In [ ]:
# Epochs: number of iterations over the entire training dataset
epochs = 50
# Number of iterations between printing loss and accuracy 
print_steps = 10

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Save

In [ ]:
def save_checkpoint(state, is_best, filename = "/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/checkpointcrazy6.pth.tar"):
    if is_best:
        best_accuracy = state['best_accuracy']
        print ("=> Saving a new best with {:.4}% validation accuracy".format(100 * best_accuracy.numpy()[0]))
        torch.save(state, filename)  # save checkpoint
    else:
        print ("=> Validation Accuracy did not improve")

In [ ]:
if os.path.isfile(resume_weights):
    print("=> loading checkpoint '{}' ...".format(resume_weights))
    if torch.cuda:
       checkpoint = torch.load(resume_weights)
    else:
       checkpoint = torch.load(resume_weights,
                               map_location=lambda storage,
                               loc: storage)
    start_epoch = checkpoint['epoch']
    best_accuracy = checkpoint['best_accuracy']
    print('=> loaded checkpoint {} (trained for {} epochs) with {:.4}% validation accuracy'.format(resume_weights,
        checkpoint['epoch'],100 * best_accuracy.numpy()[0]))
    model.load_state_dict(checkpoint['state_dict'])
else:print("no") 

# Training

In [ ]:
%%time

# Initialize steps
step = 0

# Iterate over number of epochs
for e in range(epochs):
    running_loss = 0 

    # Iterate over the entire training dataset
    # one batch per iteration 
    for input_images, labels in folderloader:
        step += 1     
        
        # Enable CUDA: use GPUs for model computation
        if torch.cuda.is_available():
            inputs, labels = input_images.to('cuda'), labels.to('cuda')   
        else:
            inputs, labels = input_images, labels
        
        # Clear the gradients of all optimized tensors
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model.forward(inputs)
        _, predicted = torch.max(outputs.data, 1)
     
        #print(predicted,labels)
        validation_correct = 0
        validation_total = 0

        training_correct = 0
        training_total = 0        

        # no_grad() prevents tracking history (and using memory)
        with torch.no_grad(): 
            
            # Iterate over the entire validation dataset
            for input_images3, labels3 in folderloader:
                
                # Enable CUDA: use GPUs for model computation
                if torch.cuda.is_available():
                    input_images3, labels3 = input_images3.to('cuda'), labels3.to('cuda')
                
                
                # Make predictions 
                outputs3 = model(input_images3)
                _, predicted3 = torch.max(outputs3.data, 1)
                #print(predicted2,labels2)
                
                # Count total and correct predictions
                training_total += labels3.size(0)
                training_correct += (predicted3 == labels3).sum().item()   

        with torch.no_grad(): 
            
            # Iterate over the entire validation dataset
            for input_images2, labels2 in folderloader_valid:
                
                # Enable CUDA: use GPUs for model computation
                if torch.cuda.is_available():
                    input_images2, labels2 = input_images2.to('cuda'), labels2.to('cuda')
                
                # Make predictions 
                outputs2 = model(input_images2)
                _, predicted2 = torch.max(outputs2.data, 1)
                #print(predicted2,labels2)
                
                # Count total and correct predictions
                validation_total += labels2.size(0)
                validation_correct += (predicted2 == labels2).sum().item()
        loss = loss_function(outputs, labels)
        val_acc = validation_correct / validation_total
        train_acc = training_correct / training_total
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate and print running training loss
        running_loss += loss.item()       
        if step % print_steps == 0:
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "| Loss: {:.4f}".format(running_loss/print_steps),
                  "| Val Acc: {:.4f}".format(val_acc),
                  "| Train Acc: {:.4f}".format(train_acc),)
            running_loss = 0
    acc1 = torch.FloatTensor([val_acc])
    is_best = bool(acc1.numpy() > best_accuracy.numpy())
    best_accuracy = torch.FloatTensor(max(acc1.numpy(), best_accuracy.numpy()))
    save_checkpoint({
        'epoch' : start_epoch + epochs,
        'state_dict' : model.state_dict(),
        'best_accuracy' : best_accuracy
        }, is_best)
        
print('\nCount of data batches seen:  {} \n'.format(step))

Epoch: 1/50...  | Loss: 0.3445 | Val Acc: 0.8661 | Train Acc: 0.8817
Epoch: 1/50...  | Loss: 0.2020 | Val Acc: 0.8571 | Train Acc: 0.9576
=> Validation Accuracy did not improve
Epoch: 2/50...  | Loss: 0.0359 | Val Acc: 0.8571 | Train Acc: 0.9643
Epoch: 2/50...  | Loss: 0.1229 | Val Acc: 0.9018 | Train Acc: 0.9643
Epoch: 2/50...  | Loss: 0.0854 | Val Acc: 0.9018 | Train Acc: 0.9754
=> Saving a new best with 87.5% validation accuracy
Epoch: 3/50...  | Loss: 0.0272 | Val Acc: 0.8661 | Train Acc: 0.9531
Epoch: 3/50...  | Loss: 0.1257 | Val Acc: 0.8839 | Train Acc: 0.9710
Epoch: 3/50...  | Loss: 0.0791 | Val Acc: 0.9107 | Train Acc: 0.9844
=> Saving a new best with 92.86% validation accuracy
Epoch: 4/50...  | Loss: 0.0459 | Val Acc: 0.9196 | Train Acc: 0.9688
Epoch: 4/50...  | Loss: 0.1141 | Val Acc: 0.9196 | Train Acc: 0.9732
Epoch: 4/50...  | Loss: 0.0661 | Val Acc: 0.9375 | Train Acc: 0.9754
=> Validation Accuracy did not improve
Epoch: 5/50...  | Loss: 0.0488 | Val Acc: 0.9286 | Train A

# Validation and Predictions

In [ ]:
validation_correct = 0
validation_total = 0
seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
# no_grad() prevents tracking history (and using memory)
model.eval()
with torch.no_grad(): 
    
    # Iterate over the entire validation dataset
    for input_images, labels in folderloader_valid:
        
        # Enable CUDA: use GPUs for model computation
        if torch.cuda.is_available():
            input_images, labels = input_images.to('cuda'), labels.to('cuda')
        
        # Make predictions 
        outputs = model(input_images)
        _, predicted = torch.max(outputs.data, 1)
        print(input_images.size(),labels)
        
        # Count total and correct predictions
        validation_total += labels.size(0)
        validation_correct += (predicted == labels).sum().item()
# Print validation accuracy
print('Validation accuracy ({0:d} validation images): {1:.1%}'
      .format(validation_total, validation_correct / validation_total))        

In [ ]:
import torch
from torchvision import datasets
from os import listdir
from os.path import isfile, join
import os
import glob
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

data_dir = "/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set"
dataFromFolders_test = ImageFolderWithPaths(root='/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set',transform = test_transform)
folderloader_test = DataLoader(dataFromFolders_test,batch_size=1)
model.eval()
final=[]
with torch.no_grad(): 
    for input_images, labels, paths in folderloader_test:
        if torch.cuda.is_available():
            input_images, labels = input_images.to('cuda'), labels.to('cuda')

        outputs = model(input_images)
        _, predicted = torch.max(outputs.data, 1)
        final.append(predicted.cpu().numpy()[0])

my_dict = {0:1, 1:2, 2:3, 3:7, 4:4, 5:6, 6:5}
answers = np.vectorize(my_dict.get)(final).tolist()
answers = [str(x).strip(' ') for x in answers]
answers = list(map(int, answers)) 
#print(answers)
mypath = '/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set'
fileslist = []
for root, directories, files in os.walk(mypath, topdown=False):
    #print(files)
    for name in files:
        #print(name);
        if name[4:] == '.jpg':
            fileslist.append(int(name[:4]))
fileslist = np.sort(fileslist)
#print(fileslist)
arr1 = np.array(fileslist)
arr2 = np.array(answers)
b = np.stack((arr1,arr2), axis=1)

b = [[x for x in y if x != ''] for y in b]
print(b)
np.savetxt("foo.csv", b, delimiter=',', header="ImageID,Label", fmt='%1d',comments='')
# 1 - Basketball court 2 - Beach 3 - Forest 4 - Railway 
# 5 - Tennis court 6 - Water pool 7 - Others      

# waste

In [ ]:
dataiter = iter(folderloader_test)

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
matplotlib_imshow(img_grid, one_channel=True)


In [ ]:
import torch
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

# EXAMPLE USAGE:
# instantiate the dataset and dataloader
data_dir = "/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set"
dataset = ImageFolderWithPaths(data_dir,transform = test_transform) # our custom dataset
dataloader = DataLoader(dataset)
final=[]
# iterate over data
for input_images, labels, paths in dataloader:
    # use the above variables freely
    #print( paths)
    if torch.cuda.is_available():
            input_images, labels = input_images.to('cuda'), labels.to('cuda')
            #print(labels)
        
    # Make predictions 
    print((input_images).size())
    import tensorflow as tf
    import numpy as np

    # x = tf.placeholder(tf.float32, (None, 3, 256, 256))

    # reduced = tf.expand_dims(tf.reduce_mean(x, axis=1), 1)
    # reduced = tf.expand_dims(tf.reduce_mean(reduced, axis=2), 2)

    # data = input_images

    # with tf.Session() as sess:
    #     evaled = reduced.eval({x:data})
    #     print(evaled.shape) # (16, 1, 1, 64)
    outputs = model(input_images)
    _, predicted = torch.max(outputs.data, 1)
    sample_fname, _ = folderloader_test.dataset.samples[i]
    #print(sample_fname)
    #print(predicted)
    final += (predicted.cpu().data.numpy().tolist())
my_dict = {0:1, 1:2, 2:3, 3:7, 4:4, 5:6, 6:5}
answers = np.vectorize(my_dict.get)(final).tolist()
answers = [str(x).strip(' ') for x in answers]
answers = list(map(int, answers)) 
print(answers)

In [ ]:
final = []
# seed = 1
# torch.manual_seed(seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(seed)
print(dataFromFolders_test.imgs)
with torch.no_grad(): 
    
    # Iterate over the entire validation dataset
    for i, (images, labels) in enumerate(folderloader_test, 0):
    #for input_images, labels in folderloader_test:
        
        # Enable CUDA: use GPUs for model computation
        if torch.cuda.is_available():
            input_images, labels = input_images.to('cuda'), labels.to('cuda')
            print(labels)
        
        # Make predictions 
        outputs = model(input_images)
        _, predicted = torch.max(outputs.data, 1)
        sample_fname, _ = folderloader_test.dataset.samples[i]
        #print(sample_fname)
        print(predicted)
        final += (predicted.cpu().data.numpy().tolist())
my_dict = {0:1, 1:2, 2:3, 3:7, 4:4, 5:6, 6:5}
answers = np.vectorize(my_dict.get)(final).tolist()
answers = [str(x).strip(' ') for x in answers]
answers = list(map(int, answers)) 
print(answers)
#print(final)

In [ ]:
from os import listdir
from os.path import isfile, join
import os
import glob
mypath = '/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set'
# for file_name in glob.iglob('/content/drive/My Drive/IITB Sem5/GNR 638/kaggle/test_set/**/*', recursive=True):
#   print(file_name)
fileslist = []
for root, directories, files in os.walk(mypath, topdown=False):
	for name in files:
		fileslist.append(int(name[:4]))
fileslist = np.sort(fileslist)
#print(fileslist)
arr1 = np.array(fileslist)
arr2 = np.array(answers)
b = np.stack((arr1,arr2), axis=1)

b = [[x for x in y if x != ''] for y in b]
print(b)
np.savetxt("foo.csv", b, delimiter=',', header="ImageID,Label", fmt='%1d',comments='')